## Import all needed packages

In [7]:
import xarray as xr
from pyproj import CRS
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, box, LineString
import folium
import os
from datetime import datetime
import matplotlib.pyplot as plt
import altair as alt
import numpy as np
import re
import psutil
import sys
import regionmask
import cf_xarray
from pyproj import CRS
import dask.array as da


## Explore L2W datasets (with clorophyll)

#### Explore single file

In [8]:
# Specify the path to your NetCDF file
file_path = r'P:\11209243-eo\Window_extraction\INPUT\L2W_V2\L2W\S2A_MSI_2015_07_16_10_50_24_merged_westerschelde_L2W.nc'

# Open the NetCDF file with xarray
ds = xr.open_dataset(file_path)

# Display the dataset information
variable_names = list(ds.variables)
crs_wkt = ds.transverse_mercator.attrs['crs_wkt']
crs = CRS.from_string(crs_wkt)
epsg_code = crs.to_epsg()

print(variable_names)
print(f"EPSG code: {epsg_code}")

ds

['transverse_mercator', 'x', 'y', 'lon', 'lat', 'l2_flags', 'chl_re_gons']
EPSG code: 32631


<xarray.Dataset>
Dimensions:              (x: 1285, y: 663)
Coordinates:
  * x                    (x) float64 5.208e+05 5.209e+05 ... 5.978e+05 5.979e+05
  * y                    (y) float64 5.712e+06 5.712e+06 ... 5.672e+06 5.672e+06
Data variables:
    transverse_mercator  float64 ...
    lon                  (y, x) float32 ...
    lat                  (y, x) float32 ...
    l2_flags             (y, x) int32 ...
    chl_re_gons          (y, x) float32 ...
Attributes: (12/381)
    generated_by:                              ACOLITE
    generated_on:                              2024-01-18 10:07:11 CET
    contact:                                   Quinten Vanhellemont
    product_type:                              NetCDF
    metadata_profile:                          beam
    metadata_version:                          0.5
    ...                                        ...
    luts:                                      ['ACOLITE-LUT-202110-MOD1', 'A...
    luts_pressures:                            [ 500  750 1013 1100]
    slicing:                                   False
    scene_download:                            False
    runid:                                     20240118_100708
    inputfile:                                 ['/eodc/private/deltares/INPUT...

#### Explore multiple files and extracts pixel windows for coordinates defined in a Stations.xlsx file produced in "Managing coordinates for window extraction" section

#### Define functions

In [9]:
# Define a function to evaluate RAM memory usage.
def print_memory():
    # Get memory information
    memory_info = psutil.virtual_memory()
    print(f"Memory Usage: {memory_info.percent:.2f}%")


# Define a function to extract the date from the path. Data about the year.
def extract_date(path):

    match = re.search(r'(\d{4}_\d{2}_\d{2})', path)
    if match:
        return match.group()
    else:
        return "0000_00_00"  # Return a default date if no date is found
    
# Define function to get the coordinates in the netcdfs closer to the stations coordinates.
def get_exact_coordinates_in_netcdf(dataset, x_coord, y_coord):

    time_series   = dataset.sel(x=x_coord, y=y_coord, method='nearest')
    y_coord_exact = time_series['y'].values.tolist()
    x_coord_exact = time_series['x'].values.tolist()

    return y_coord_exact, x_coord_exact 

# Define function to extract pixel values
def get_window(x_coord, y_coord, stationID, dataset):

   # Get exact coordinates found in the netcdf files.
    y_coord_exact, x_coord_exact = get_exact_coordinates_in_netcdf(dataset, x_coord, y_coord)

    # Get index values or given coordinates
    y_index = np.where(dataset['y'] == y_coord_exact)[0].tolist()[0]
    x_index = np.where(dataset['x'] == x_coord_exact)[0].tolist()[0]

    # Obtaining the central pixel's longitud and latitude.
    central_lon  = x_index
    central_lat  = y_index

    # Get pixel coordinates around the central pixel
    lat_start, lat_end = central_lat - 1, central_lat + 2
    lon_start, lon_end = central_lon - 1, central_lon + 2

    # Select only data inside the window of sample. Return a filtered netcdf (dataset)
    window_values = dataset.isel(y=slice(lat_start, lat_end), x=slice(lon_start, lon_end))
    window_values = window_values.expand_dims(station=[stationID])
    
    return window_values

def get_files_and_variable_names(folder_path):
    # List all files in the folder
    files_in_folder = os.listdir(folder_path)

    # Filter the list to include only files ending with "ic.nc"
    filtered_files = [file for file in files_in_folder if file.endswith("L2W.nc")]

    # Sort the paths based on the extracted date
    sorted_files = sorted(filtered_files, key=extract_date)

    variables_to_remove = ['transverse_mercator', 'x', 'y', 'lon', 'lat']

    #TODO: This component should read both a file created with a Sentinel A and Sentinel B sensors to work. 
    #TODO: The files were manually selected and will not work for other cases in in the same sensor utput is selected.

    ds_start = xr.open_dataset(os.path.join(folder_path, sorted_files[0]))
    ds_end   = xr.open_dataset(os.path.join(folder_path, sorted_files[-1]))

    ds_start = ds_start.drop_vars(variables_to_remove)
    ds_end   = ds_end.drop_vars(variables_to_remove)

    variable_names_start = list(ds_start.variables)
    variable_names_end   = list(ds_end.variables)

    print("The variables in SA derived products are:",variable_names_start)
    print("The variables in SB derived products are:",variable_names_end)

    variable_names = list(set(variable_names_start + variable_names_end))
    print("The variables", variable_names)

    return sorted_files, variable_names

#### Execute functions

In [10]:
# Define input and outputs

# folder_path_l2w_data = r'P:\11209243-eo\Window_extraction\INPUT\L2W'
folder_path_l2w_data   = r'P:\11209243-eo\Window_extraction\INPUT\L2W_V2\L2W'
file_path_stations     = r'P:\11209243-eo\Window_extraction\OUTPUT\Stations.xlsx'

file_path_output_nc    = r'P:\11209243-eo\Window_extraction\OUTPUT\window_extraction_output.nc'
file_path_output_excel = r'P:\11209243-eo\Window_extraction\OUTPUT\window_extraction_output.xlsx' 

In [11]:
# Get files and variable names for S2A and S2B sensors 
sorted_files, variable_names = get_files_and_variable_names(folder_path_l2w_data)

# Define subset for testing. Few images and few variables to read
sorted_files = sorted_files[:]
variable_names =variable_names[:]

#variable_names = ['l2_flags', 'chl_re_gons']
variable_names = ['chl_re_gons']

df_list = []
ds_list = []

df_stations = pd.read_excel(file_path_stations)

for variable_name in variable_names:

    print('\nCalculated variable: '+ variable_name)
    station_windows_list = []

    for file in sorted_files:
        file_path = os.path.join(folder_path_l2w_data, file)
        print("Processing file: ", file_path)
        with xr.open_dataset(file_path, chunks={'x': 500, 'y': 500}) as dataset:
            time_series = [datetime.fromisoformat(dataset.attrs.get("isodate"))] 
            
            try:
                dataset = xr.concat([dataset[variable_name]], dim=xr.DataArray(time_series, coords={"time": time_series}, dims=["time"]))
                
                for index, row in df_stations.iterrows():
                    station_window = get_window(row['x_coord'], row['y_coord'], row['stationID'],dataset)
                    station_windows_list.append(station_window)
                    del station_window    
                    
                print(f"Memory usage of dataset: {sys.getsizeof(dataset)} bytes")
                print_memory()   
                del dataset 

            except Exception as e:
                print(f"The file {file}: {e} does not have the current variable. It will be skipped")
                continue  

    # Create a single netcdf with all the data
    merged_stations = xr.concat(station_windows_list, dim='station').groupby('station').max(dim='station')
    del station_windows_list

    ds_list.append(merged_stations)

# Create a single netcdf
merged_ds = xr.merge(ds_list)
merged_ds['time'] = merged_ds['time'].astype('datetime64')
merged_ds.to_netcdf(file_path_output_nc)

The variables in SA derived products are: ['l2_flags', 'chl_re_gons']
The variables in SB derived products are: ['l2_flags', 'chl_re_gons']
The variables ['l2_flags', 'chl_re_gons']

Calculated variable: chl_re_gons
Processing file:  P:\11209243-eo\Window_extraction\INPUT\L2W_V2\L2W\S2A_MSI_2015_07_16_10_50_24_merged_westerschelde_L2W.nc
Memory usage of dataset: 96 bytes
Memory Usage: 43.30%
Processing file:  P:\11209243-eo\Window_extraction\INPUT\L2W_V2\L2W\S2A_MSI_2023_08_23_10_56_47_merged_westerschelde_L2W.nc
Memory usage of dataset: 96 bytes
Memory Usage: 43.20%


C:\Users\fuentesm\AppData\Local\Temp\ipykernel_1016\1626129040.py:51: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  merged_ds['time'] = merged_ds['time'].astype('datetime64')


In [12]:
# Open the NetCDF file with xarray
ds_processed = xr.open_dataset(file_path_output_nc)
ds_processed

<xarray.Dataset>
Dimensions:      (x: 29, y: 32, time: 2, station: 12)
Coordinates:
  * x            (x) float64 5.214e+05 5.215e+05 ... 5.871e+05 5.872e+05
  * y            (y) float64 5.689e+06 5.689e+06 ... 5.711e+06 5.711e+06
  * time         (time) datetime64[ns] 2015-07-16T10:50:24.456000 2023-08-23T...
  * station      (station) int32 0 1 2 3 4 5 6 7 8 9 10 11
Data variables:
    chl_re_gons  (station, time, y, x) float32 ...
Attributes: (12/15)
    long_name:      chlorophyll a concentration in mg m-3 derived from red-edge
    parameter:      chl_re_gons
    waves:          [665 704 783]
    algorithm:      chl_re_gons
    astar_chl:      0.015
    rededge_band:   704.0
    ...             ...
    red_band:       664.0
    units:          mg m-3
    reference:      Gons et al. 1999/2005
    grid_mapping:   transverse_mercator
    chl_coef:       [1.61  0.082 0.6   0.7   0.4   1.05 ]
    validity:       [0.005 0.63 ]